# Debug Drill: The Silent Degradation

**Scenario:**
Your churn model has been in production for 3 months. Business metrics are declining.

"Our retention offers aren't working anymore!" the PM complains. "Customers aren't responding."

Nobody noticed the model was degrading because there were no alerts.

**Your Task:**
1. Identify where the model degraded
2. Set up proper monitoring with alerts
3. Create a retraining decision framework
4. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from typing import Dict, List, Tuple

np.random.seed(42)

In [ ]:
def generate_data(n_samples, drift_factor=0, concept_drift=0):
    """Generate data with optional drift."""
    tenure = np.random.exponential(20, n_samples)
    spend = np.random.normal(100, 30, n_samples)
    tickets = np.random.poisson(2, n_samples)
    
    # Apply data drift
    if drift_factor > 0:
        spend = spend * (1 + drift_factor * 0.5)
        tenure = tenure * (1 + drift_factor * 0.3)
    
    # Base churn probability
    churn_prob = 0.2 + 0.2 * (tickets > 3) - 0.1 * (spend > 100)
    
    # Apply concept drift (relationship changes!)
    if concept_drift > 0:
        # Now spend becomes MORE predictive, tickets less
        churn_prob = 0.2 - 0.2 * (spend > 100) + 0.05 * (tickets > 3)
    
    churn_prob = np.clip(churn_prob, 0.05, 0.95)
    churned = (np.random.random(n_samples) < churn_prob).astype(int)
    
    return pd.DataFrame({
        'tenure': tenure,
        'spend': spend,
        'tickets': tickets,
        'churned': churned
    })

# Generate training data (no drift)
train_data = generate_data(2000)

# Train model
features = ['tenure', 'spend', 'tickets']
X_train = train_data[features]
y_train = train_data['churned']

model = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)
model.fit(X_train, y_train)

baseline_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print(f"Baseline AUC: {baseline_auc:.3f}")

In [ ]:
# ===== COLLEAGUE'S CODE (BUG: No monitoring!) =====

def broken_predict(model, data, features):
    """Production prediction without any monitoring."""
    X = data[features]
    predictions = model.predict_proba(X)[:, 1]
    return predictions  # Just returns predictions, no monitoring!

# Simulate 12 weeks of production
print("=== Production Over 12 Weeks (No Monitoring) ===")
weekly_data = []

for week in range(12):
    # Gradual drift increases over time
    drift = min(week / 8, 1.0) * 0.6
    concept_drift = max(0, (week - 4) / 8) * 0.8
    
    prod_data = generate_data(500, drift_factor=drift, concept_drift=concept_drift)
    
    # Make predictions (without monitoring)
    preds = broken_predict(model, prod_data, features)
    
    # Calculate actual performance (we can see this in hindsight)
    actual_auc = roc_auc_score(prod_data['churned'], preds)
    
    weekly_data.append({
        'week': week + 1,
        'auc': actual_auc,
        'drift': drift,
        'concept_drift': concept_drift
    })
    
    # BUG: No alerts triggered, even when performance drops!
    if week == 11:
        print(f"Week 12: AUC = {actual_auc:.3f} (down from {baseline_auc:.3f})")
        print("❌ Nobody was alerted! Business metrics are already suffering.")

In [ ]:
# Visualize the degradation
df_weekly = pd.DataFrame(weekly_data)

plt.figure(figsize=(10, 5))
plt.plot(df_weekly['week'], df_weekly['auc'], 'b-o', linewidth=2, markersize=8)
plt.axhline(y=baseline_auc, color='g', linestyle='--', label='Baseline')
plt.axhline(y=baseline_auc - 0.05, color='orange', linestyle='--', label='Warning')
plt.axhline(y=baseline_auc - 0.10, color='r', linestyle='--', label='Critical')

# Mark when alerts SHOULD have fired
for _, row in df_weekly.iterrows():
    if row['auc'] < baseline_auc - 0.10:
        plt.scatter(row['week'], row['auc'], color='red', s=200, zorder=5)

plt.xlabel('Week')
plt.ylabel('AUC')
plt.title('Model Degradation (Undetected)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n🚨 Red dots show weeks where CRITICAL alerts should have fired!")

---

## Your Investigation

### Step 1: Understand the failure

In [ ]:
print("=== Silent Degradation Analysis ===")
print()
print("What happened:")
print("  Week 1-4: Model performing well (AUC ~0.70)")
print("  Week 5-8: Data drift starts, concept drift begins")
print("  Week 9-12: Significant degradation (AUC ~0.55)")
print()
print("Why no one noticed:")
print("  - No monitoring in place")
print("  - No alerts configured")
print("  - No drift detection")
print()
print("Business impact:")
print("  - Targeting wrong customers with retention offers")
print("  - Wasting marketing budget")
print("  - Missing actual churners")

### Step 2: TODO - Implement monitoring

In [ ]:
# TODO: Create monitoring class

# Uncomment and complete:

# def calculate_psi(expected, actual, buckets=10):
#     """Calculate Population Stability Index."""
#     bins = np.percentile(expected, np.linspace(0, 100, buckets + 1))
#     bins[0], bins[-1] = -np.inf, np.inf
#     
#     exp_counts = np.histogram(expected, bins)[0] / len(expected)
#     act_counts = np.histogram(actual, bins)[0] / len(actual)
#     
#     exp_counts = np.clip(exp_counts, 0.0001, None)
#     act_counts = np.clip(act_counts, 0.0001, None)
#     
#     return np.sum((act_counts - exp_counts) * np.log(act_counts / exp_counts))
# 
# class ModelMonitor:
#     def __init__(self, baseline_data, baseline_auc, thresholds):
#         self.baseline_data = baseline_data
#         self.baseline_auc = baseline_auc
#         self.thresholds = thresholds
#         self.alerts = []
#     
#     def check(self, current_data, predictions, actuals=None):
#         """Check for drift and performance degradation."""
#         alerts = []
#         
#         # Check data drift (PSI)
#         for feature in ['tenure', 'spend', 'tickets']:
#             psi = calculate_psi(
#                 self.baseline_data[feature].values,
#                 current_data[feature].values
#             )
#             if psi > self.thresholds['psi_critical']:
#                 alerts.append(f"🚨 CRITICAL: {feature} PSI={psi:.2f}")
#             elif psi > self.thresholds['psi_warning']:
#                 alerts.append(f"⚠️ WARNING: {feature} PSI={psi:.2f}")
#         
#         # Check performance (if we have ground truth)
#         if actuals is not None:
#             current_auc = roc_auc_score(actuals, predictions)
#             auc_drop = self.baseline_auc - current_auc
#             
#             if auc_drop > self.thresholds['auc_critical']:
#                 alerts.append(f"🚨 CRITICAL: AUC dropped {auc_drop:.2%}")
#             elif auc_drop > self.thresholds['auc_warning']:
#                 alerts.append(f"⚠️ WARNING: AUC dropped {auc_drop:.2%}")
#         
#         self.alerts.extend(alerts)
#         return alerts
# 
# # Configure thresholds
# thresholds = {
#     'psi_warning': 0.1,
#     'psi_critical': 0.25,
#     'auc_warning': 0.03,
#     'auc_critical': 0.05
# }
# 
# monitor = ModelMonitor(train_data, baseline_auc, thresholds)
# print("✓ ModelMonitor created")

In [ ]:
# TODO: Re-run with monitoring

# Uncomment:

# print("=== Re-simulating with Monitoring ===")
# monitor.alerts = []  # Reset alerts
# 
# for week in range(12):
#     drift = min(week / 8, 1.0) * 0.6
#     concept_drift = max(0, (week - 4) / 8) * 0.8
#     
#     prod_data = generate_data(500, drift_factor=drift, concept_drift=concept_drift)
#     preds = model.predict_proba(prod_data[features])[:, 1]
#     
#     alerts = monitor.check(prod_data, preds, prod_data['churned'])
#     
#     if alerts:
#         print(f"\nWeek {week + 1}:")
#         for alert in alerts:
#             print(f"  {alert}")
# 
# print(f"\n✓ Monitoring caught {len(monitor.alerts)} alerts!")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert len(monitor.alerts) > 0, "Monitor should have caught alerts"
# assert any('CRITICAL' in a for a in monitor.alerts), "Should have critical alerts"
# 
# print("✓ Monitoring system working!")
# print(f"✓ Total alerts: {len(monitor.alerts)}")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Silent Degradation

### What happened:
- (Your answer: How long did degradation go unnoticed?)

### Root cause:
- (Your answer: Why were there no alerts?)

### How to prevent:
- (Your answer: What monitoring should be in place?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Monitor early, alert often.** Don't wait for business metrics to drop.

2. **Track both drift and performance.** PSI for features, AUC for accuracy.

3. **Set tiered thresholds.** Warning → investigate, Critical → act.

4. **Automate retraining decisions.** Define clear triggers.

---

## Monitoring Checklist

| Metric | Warning | Critical |
|--------|---------|----------|
| PSI (data drift) | > 0.1 | > 0.25 |
| AUC drop | > 3% | > 5% |
| Prediction shift | > 1 std | > 2 std |